In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import re

In [2]:
from nltk.corpus import stopwords
stopwords.words('english')

stopwords_list= stopwords.words('english')
stopwords_list.append('the')


In [4]:
target_wiki_path ='/Users/areej/Desktop/wiki_extract/wiki_subset_for_topic_matching.csv'


# wiki_extracted folder contains wikipedia dump extacted into csv files
# using https://github.com/attardi/wikiextractor
source_wiki_extractor ='/Users/areej/Desktop/wiki_extracted/'

In [39]:
# extract around 1M wiki docs
# extract from wikipedia extracted using wiki_extractor
from bz2 import BZ2File



import os
import sys
import csv
import json
import time

line_count=0
with open(target_wiki_path, 'w' , encoding='utf8') as out:
    writer = csv.writer(out, dialect='excel')
    for subdir, dirs, files in os.walk(source_wiki_extractor):
        for filename in files:
            filepath = subdir + os.sep + filename
            if line_count > 1000000:
                break
            else:
                if filename.endswith(".bz2") :
                    #print("bz2 file reader:", filename)
                    file = BZ2File(filepath)
                    with file as f:
                        for line in f: 
                            article_json = json.loads(line)
                            title, text = article_json["title"],article_json["text"]
                            #print("title:\n", title)
                            text= ' '.join(text.split('\n')[1:])

                            doc_words= text.split()

                            if  len(doc_words)> 200:
                                doc= ' '.join(doc_words)
                                writer.writerow([clean(title),  (doc)])
                                line_count+=1
                                if line_count% 1000 == 0:
                                    print(line_count)
                
print("Number of docs ", line_count, 'are written and saved')

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [12]:
# read wiki_full_doc file
wiki_subset=pd.read_csv(wiki_path, names=['title', 'article'])
wiki_subset= wiki_subset['article'].to_numpy()
len(wiki_subset)

1005280

In [3]:
# get bhatia's topics and labels

bhatia_topics_path='/data/topics_raw/bhatia_data_whole.csv'

bhatia_data=pd.read_csv(bhatia_topics_path)
bhatia_data

,Unnamed: 0,topic_id,terms,label,rating
0,0,0,"vmware,server,virtual,oracle,update,virtualiza...",cloud computing,2.333333
1,1,0,"vmware,server,virtual,oracle,update,virtualiza...",hypervisor,1.200000
2,2,0,"vmware,server,virtual,oracle,update,virtualiza...",microsoft exchange server,2.333333
3,3,0,"vmware,server,virtual,oracle,update,virtualiza...",vmware,2.000000
4,4,0,"vmware,server,virtual,oracle,update,virtualiza...",windows 2000,1.666667
...,...,...,...,...,...
4327,4327,227,"exposure,concentration,water,soil,metal,study,...",water quality,1.714286
4328,4328,227,"exposure,concentration,water,soil,metal,study,...",aerosol,1.000000
4329,4329,227,"exposure,concentration,water,soil,metal,study,...",ammonia,0.666667
4330,4330,227,"exposure,concentration,water,soil,metal,study,...",carbon dioxide,1.000000


In [5]:
# create gold data (only labels with rating above 2 are kept)
bhatia_data_filtered= bhatia_data.loc[(bhatia_data['rating'] >= 2)]
len(bhatia_data_filtered)                              

1156

In [6]:
bhatia_data_filtered

,Unnamed: 0,topic_id,terms,label,rating
0,0,0,"vmware,server,virtual,oracle,update,virtualiza...",cloud computing,2.333333
2,2,0,"vmware,server,virtual,oracle,update,virtualiza...",microsoft exchange server,2.333333
3,3,0,"vmware,server,virtual,oracle,update,virtualiza...",vmware,2.000000
8,8,0,"vmware,server,virtual,oracle,update,virtualiza...",web application,2.125000
12,12,0,"vmware,server,virtual,oracle,update,virtualiza...",virtualization,2.428571
...,...,...,...,...,...
4301,4301,226,"model,datum,estimate,method,predict,parameter,...",parameter,2.142857
4307,4307,226,"model,datum,estimate,method,predict,parameter,...",regression analysis,2.142857
4309,4309,226,"model,datum,estimate,method,predict,parameter,...",estimation,2.000000
4313,4313,227,"exposure,concentration,water,soil,metal,study,...",ozone,2.000000


In [115]:
for index, i in bhatia_data_filtered.iterrows():
    print(index, 'terms:',i['terms'], 'label:', i['label'], ' rate:', i['rating'])

0 terms: vmware,server,virtual,oracle,update,virtualization,application,infrastructure,management,microsoft label: cloud computing  rate: 2.33333333333
2 terms: vmware,server,virtual,oracle,update,virtualization,application,infrastructure,management,microsoft label: microsoft exchange server  rate: 2.33333333333
3 terms: vmware,server,virtual,oracle,update,virtualization,application,infrastructure,management,microsoft label: vmware  rate: 2.0
8 terms: vmware,server,virtual,oracle,update,virtualization,application,infrastructure,management,microsoft label: web application  rate: 2.125
12 terms: vmware,server,virtual,oracle,update,virtualization,application,infrastructure,management,microsoft label: virtualization  rate: 2.4285714285699997
18 terms: vmware,server,virtual,oracle,update,virtualization,application,infrastructure,management,microsoft label: operating system  rate: 2.1111111111099996
23 terms: obama,mccain,campaign,john,barack,president,senator,candidate,convention,clinton la

906 terms: plant,tree,fruit,grow,soil,foot,leaf,seed,root,flower label: inflorescence  rate: 2.25
911 terms: plant,tree,fruit,grow,soil,foot,leaf,seed,root,flower label: seedling  rate: 2.44444444444
912 terms: god,church,christian,thing,word,christ,religion,divine,sin,sermon label: jesus  rate: 2.4
914 terms: god,church,christian,thing,word,christ,religion,divine,sin,sermon label: god the father  rate: 3.0
915 terms: god,church,christian,thing,word,christ,religion,divine,sin,sermon label: christian theology  rate: 2.1428571428599996
916 terms: god,church,christian,thing,word,christ,religion,divine,sin,sermon label: faith  rate: 2.2
918 terms: god,church,christian,thing,word,christ,religion,divine,sin,sermon label: salvation  rate: 2.0
919 terms: god,church,christian,thing,word,christ,religion,divine,sin,sermon label: eternal life (christianity)  rate: 2.0
923 terms: god,church,christian,thing,word,christ,religion,divine,sin,sermon label: son of god  rate: 2.2857142857099997
925 terms:

1472 terms: portrait,lady,collection,sir,duke,queen,john,charles,head,william label: elizabeth  rate: 2.0
1484 terms: play,poet,story,character,theatre,stage,write,poetry,drama,poem label: poetry  rate: 2.1428571428599996
1486 terms: play,poet,story,character,theatre,stage,write,poetry,drama,poem label: english literature  rate: 2.4285714285699997
1495 terms: play,poet,story,character,theatre,stage,write,poetry,drama,poem label: writer  rate: 2.33333333333
1499 terms: play,poet,story,character,theatre,stage,write,poetry,drama,poem label: novel  rate: 2.16666666667
1506 terms: water,air,experiment,wire,heat,produce,temperature,tube,mr.,electricity label: heat sink  rate: 2.0
1510 terms: water,air,experiment,wire,heat,produce,temperature,tube,mr.,electricity label: thermal expansion  rate: 2.0
1512 terms: water,air,experiment,wire,heat,produce,temperature,tube,mr.,electricity label: electric current  rate: 2.33333333333
1513 terms: water,air,experiment,wire,heat,produce,temperature,tube,

2283 terms: government,political,country,leader,president,power,party,minister,democracy,protest label: democratic party (united states)  rate: 2.33333333333
2286 terms: government,political,country,leader,president,power,party,minister,democracy,protest label: political party  rate: 2.5
2287 terms: government,political,country,leader,president,power,party,minister,democracy,protest label: prime minister  rate: 2.1428571428599996
2290 terms: government,political,country,leader,president,power,party,minister,democracy,protest label: politics  rate: 2.7142857142900003
2293 terms: government,political,country,leader,president,power,party,minister,democracy,protest label: government  rate: 2.77777777778
2295 terms: government,political,country,leader,president,power,party,minister,democracy,protest label: social democracy  rate: 2.16666666667
2296 terms: government,political,country,leader,president,power,party,minister,democracy,protest label: head of state  rate: 2.5
2300 terms: universi

3060 terms: dna,methylation,damage,repair,chromosome,histone,cell,gene,replication,promoter label: dna repair  rate: 2.33333333333
3064 terms: dna,methylation,damage,repair,chromosome,histone,cell,gene,replication,promoter label: dna  rate: 2.1111111111099996
3067 terms: dna,methylation,damage,repair,chromosome,histone,cell,gene,replication,promoter label: cell cycle  rate: 2.0
3072 terms: dna,methylation,damage,repair,chromosome,histone,cell,gene,replication,promoter label: rna  rate: 2.0
3073 terms: dna,methylation,damage,repair,chromosome,histone,cell,gene,replication,promoter label: dna replication  rate: 2.7142857142900003
3074 terms: dna,methylation,damage,repair,chromosome,histone,cell,gene,replication,promoter label: gene  rate: 2.0
3078 terms: tumor,cell,carcinoma,expression,cancer,tissue,tumour,malignant,invasion,growth label: colorectal cancer  rate: 2.6
3079 terms: tumor,cell,carcinoma,expression,cancer,tissue,tumour,malignant,invasion,growth label: renal cell carcinoma  ra

3821 terms: disease,review,development,recent,clinical,target,therapeutic,therapy,drug,potential label: targeted therapy  rate: 2.0
3824 terms: disease,review,development,recent,clinical,target,therapeutic,therapy,drug,potential label: chemotherapy  rate: 2.0
3827 terms: disease,review,development,recent,clinical,target,therapeutic,therapy,drug,potential label: clinical research  rate: 2.16666666667
3862 terms: score,quality,scale,measure,assess,assessment,study,questionnaire,test,result label: measurement  rate: 2.25
3863 terms: score,quality,scale,measure,assess,assessment,study,questionnaire,test,result label: test (assessment)  rate: 2.4
3865 terms: score,quality,scale,measure,assess,assessment,study,questionnaire,test,result label: predictive validity  rate: 2.0
3869 terms: score,quality,scale,measure,assess,assessment,study,questionnaire,test,result label: program evaluation  rate: 2.0
3870 terms: score,quality,scale,measure,assess,assessment,study,questionnaire,test,result label

# write bhatia_data_filtered to file folder_name bhatia_topics_filtered

In [51]:
import csv
terms_labels={}
for terms, label in zip(bhatia_data_filtered['terms'], bhatia_data_filtered['label']):
    if terms not in terms_labels:
        terms_labels[terms]= label
    else:
        terms_labels[terms]= terms_labels[terms]+','+label
print(len(terms_labels))
#save to folder in csv file
with open('/data/bhatia_topics_filtered/bhatia_topics_filtered.csv','w') as csv_file:
    csv_writer= csv.writer(csv_file)
    for terms, labels in terms_labels.items():
        print(labels)
        print("---")
        print(terms)
        
        csv_writer.writerow([labels,terms.replace(',',' ')])

219
cloud computing,microsoft exchange server,vmware,web application,virtualization,operating system
---
vmware,server,virtual,oracle,update,virtualization,application,infrastructure,management,microsoft
conservative democrat,democratic party (united states),presidential nominee,bill clinton
---
obama,mccain,campaign,john,barack,president,senator,candidate,convention,clinton
windows 10,windows 8.1,operating system,windows 8,internet explorer,windows server 2003,windows 2000,windows 7,microsoft office,microsoft visual studio
---
file,windows,software,server,version,install,user,microsoft,application,support
pc card,operating system,flash memory,motherboard,desktop computer,personal computer,microprocessor,usb
---
drive,computer,card,laptop,memory,battery,usb,intel,processor,hard
xbox one,xbox 360,virtual console,wii,xbox (console),video game console
---
game,play,video,xbox,360,wii,gaming,player,release,nintendo
election,electoral college,by-election,general election,primary election
--

In [86]:
len(bhatia_data_filtered)

1156

In [98]:
# find topic that has been removed completly because no label with rateing above 2 was found
terms= bhatia_data['terms'].unique()
terms_f= bhatia_data_filtered['terms'].unique()
print('len terms before filter', len(terms), 'after filter', len(terms_f))
print("following terms have been removes:")
for i in terms:
    if i not in terms_f:
        print(i)

len terms before filter 228 after filter 219
following terms have been removes:
san,francisco,diego,los,california,angeles,2006,2005,conference,october
dog,animal,tree,cat,bird,water,find,fish,lake,plant
john,james,david,paul,robert,michael,bear,george,william,richard
street,ave,west,york,st.,broadway,park,avenue,east,foot
game,sox,yankee,martinez,williams,red,ball,mariners,field,run
transport,uptake,transporter,increase,effect,hypoxium,cell,barrier,study,decrease
rat,effect,animal,treatment,model,increase,administration,mg/kg,injection,induce
skin,drug,platelet,wound,delivery,formulation,study,result,topical,healing
degree,temperature,production,condition,rate,process,increase,concentration,removal,acid


# [2] match topics with documents based in word2vec


In [18]:
from gensim.models import KeyedVectors
# load word2vec
def loadWord2vec():
    embd_file= "/support_files/GoogleNews-vectors-negative300.bin.gz"
    model = KeyedVectors.load_word2vec_format(embd_file, binary=True , limit=500000)
    

    print("Done Loading Word2Vec.")
    return model

w2v = loadWord2vec()

Done Loading Word2Vec.


In [36]:
#compute each doc vector then compare (faster)
def compute_doc_vec(docs):
    num_not_found=0
    docs_vecs={}
    for doc in docs:
        doc_vectors=[]

        for word in doc.split():
            try:
                word_pro= re.sub(r'\W+', '',word)
                doc_vectors.append(w2v[word_pro])
            except: 
                try:
                    word=word.lower()
                    doc_vectors.append(w2v[re.sub(r'\W+', '',word)])
                except:
                    num_not_found+=1
        # find avg_vector by column-wise sum
        avg_vec=  np.sum(doc_vectors, axis=0)/ len(doc_vectors)
        docs_vecs[doc] = avg_vec 

        if len(docs_vecs) % 10000 == 0:
            print(len(docs_vecs))

            
            
    print("len docs_vecs",len(docs_vecs))   
    print('num not found', num_not_found)

    return docs_vecs



docs_vecs= compute_doc_vec(wiki_subset) 

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
len docs_vecs 1005255
num not found 115961363


In [69]:
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
import unicodedata
def compute_avg_vector(words, w2v):
    v=[]
    for word in words.split():
        #print(word)
        try:
            v.append(w2v[word])
        except: 
            #print('not found', word)
            pass 
    v=  np.sum(np.array(v), axis=0)/ len(v)
    return v
    
def get_doc(k, docs):
    for key, label, doc in docs:
        if key == k: return doc

        
def find_sim_doc_using_scipy(vec, docs_vecs_list, docs):
    aa = vec.reshape(1,-1) # (1,300)
    ba = docs_vecs_list
    
    r=scipy.spatial.distance.cdist(aa,ba ,'cosine')
   # print(len(r[0]))
    idx = np.argmin(r)
    return docs[idx] , r[0][idx], idx

#### tf-idf methods

def remove_accents(s):
    return ''.join((c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'))

def deepclean(doc):
    doc = doc.lower()
    doc = remove_accents(doc)
    doc = re.sub(r'\(\),[?|$|.|!]','',doc)
    doc=re.sub(r'[^a-zA-Z]',r' ',doc)
    return doc
def clean(doc):
    doc = doc.lower()
    doc = remove_accents(doc)
    #doc = re.sub(r'\(\),[?|$|.|!]','',doc)
    #doc=re.sub(r'[^a-zA-Z]',r' ',doc)
    return doc
def build_idf_dict():
    print("in build_idf_dict")
    
    idf_dict={}
    with open('/support_files/tf_idf_socres_for_wikipedia/extracted_terms.csv','r') as tfidf_file:
        first_line= tfidf_file.readline()
        print(first_line)
        for line in tfidf_file:
            
            try:
                word,freq, total_freq, idf = line.split(',')
                if word not in idf_dict:
                    idf_dict[word]=idf
                else:
                    print('error')
            except:
                pass
                #print("not splitted", line)
    return idf_dict
def get_idf_from_dict(word, d):
    idf =0
    if word in d:
        idf = d[word]
    return idf

def countWordsTFIDF(doc, stops,d):
    counts = {}
    doc_length= len(doc.split())
    doc = clean(doc)
    # 1 get word counts in doc
    for word in doc.split():
        
        if word not in stops:            
            if word in counts:
                counts[word] += 1
            else:
                counts[word] = 1
    # 2 get idf and update counts
    for word, count in counts.items():
        tf= count/ doc_length
        idf =get_idf_from_dict(word,d)
        
        
        try:
            counts[word]= tf*float(idf)
            #print(word, 'tf', tf, 'idf', idf)
        except : 
            print(word, 'tf', type(tf), tf, 'idf', type(idf),idf )
            print("error check what is the problem")
            sys.exit(0)
            counts[word]= tf
    return counts
##############################
        
def getTopnTFIDF(text,stopwords,n,d):
    counts= countWordsTFIDF(text, stopwords,d)
    words = list(counts.keys())
    words.sort(reverse=True, key=lambda v:counts[v])
    topn=[]
    try:
        topn=words[:n]
    except:
        pass
    return topn


In [54]:
docs_vecs_list= list(docs_vecs.values())
docs_list= list(docs_vecs.keys())

In [55]:
len(docs_vecs_list), len(docs_vecs)

(1005255, 1005255)

In [56]:
idf_dict=build_idf_dict()

in build_idf_dict
token,frequency,total,idf



In [7]:
len(idf_dict)

9868855

In [83]:
# match topic with wiki_doc using cosine distance then choose 20 words using tf/idf
import scipy
import time
topic_label_tfidf=[]

for terms, label in zip(bhatia_data_filtered['terms'], bhatia_data_filtered['label']):
    # find terms+label average vector from wrv
    terms= ' '.join(terms.split(',')) # to remove ','
    terms_label_vector=compute_avg_vector(terms+' '+label,w2v)   
    

    doc, sim,idx= find_sim_doc_using_scipy(terms_label_vector, docs_vecs_list, docs_list) # cosine distance
    # choose 20 words using tf/idf
    topwords_tfidf= getTopnTFIDF(deepclean(doc),stopwords_list, 40, idf_dict)
    #keep terms from topword_tfidf that do not already appear in the topic
    for term in topwords_tfidf:
        if term not in terms.split():
            terms+=' '+term
      
    topic_label_tfidf.append([terms, label])
    
    length= len(topic_label_tfidf)

    if length% 100 ==0:
        print(length)
        print('terms:',terms)
        print("label:", label)
        print(idx,doc[:200])
        print('-'*50)

        
        
    

100
terms: oil energy gas water power fuel global price plant natural lng regasification plants cold gasification turbine exhaust viable floating fluid usage conventional temperature joule acceptability argon utilisation byproducts urea cryogenic liquefied refrigeration methane extracting barges barge injected lieu towed nitrogen cooling processed converting atmospheric offshore economically heated
label: biofuel
391125 Regasification is a process of converting liquefied natural gas (LNG) at −162 °C (−260 °F) temperature back to natural gas at atmospheric temperature. LNG gasification plants can be located on land as well as on floating barges. Floating barge mounted plants have the advantage that they can be towed to new offshore locations for better usage in response to changes in the business environment. In a conventional , LNG is heated by sea water to convert it to natural gas / methane gas. In addition to regasification, many valuable industrial byproducts can be produced using 

300
terms: music musical part opera play concert sing composer song beethoven hvoslef bergen orchestra ketil concertino concerto works kvartoni perarchi piano cello compleanno flute lazarof lidholm rajna conservatory blomdahl quartet violin barabbas organ serenata ingvar antigone birger grieg solo bassoon sextet double concertos fi furthered th harald quintet clarinet
label: symphony
898516 Ketil Hvoslef (né Ketil Sæverud; born 19 July 1939 in Bergen, Norway) is a Norwegian composer. He is the son of composer Harald Sæverud. Hvoslef studied viola and organ at the Bergen Music Conservatory (now a part of Grieg Academy) graduating in 1962. He furthered his studies in Stockholm with Karl-Birger Blomdahl and Ingvar Lidholm, then in London with Thomas Rajna and Henri Lazarof. He taught music theory and ear training at the Bergen Music Conservatory (1963–1979). Hvoslef had his debut as a composer in 1964 with the "Concertino for Piano and Orchestra". He has composed works for the stage inclu

700
terms: school student education teacher high program test class board district ttuisd students texas cbes exams courses supplemental homeschool credit kindergarten accredited tech diploma th families grade teks serves traditional homebound elementary lubbock diplomas individual enroll assessments adequately middle independent university educate full special dancers aligned
label: curriculum
702315 Texas Tech University Independent School District (TTUISD) is an accredited learning school from kindergarten through twelfth grade program in Texas, offering high school diplomas. It is operated by Texas Tech University, which is located in Lubbock. TTUISD offers individual courses (supplemental), credit by exams (CBEs), homeschool curriculum, bulk testing services, and a full-time Texas Diploma Program that includes high school, middle school and elementary school. Texas Tech University Independent School District (TTUISD) was established by legislative and state board action in 1993 as

1000
terms: nerve stimulation response activity neuron frequency sensory motor potential change nervous system neurons cns pns brain neurotransmitter neural peripheral membrane spinal cells cord cell excitable information transmits involuntary cognition processes nerves human receptors voltage signaling preferences receptor synaptically neuroglia voluntary signals consists artificial glia biological processing parts neurone
label: central nervous system
416337 The following outline is provided as an overview of and topical guide to the human nervous system: Human nervous system – the part of the human body that coordinates a person's voluntary and involuntary actions and transmits signals between different parts of the body. The human nervous system consists of two main parts, the central nervous system (CNS) and the peripheral nervous system (PNS). The CNS contains the brain and spinal cord. The PNS consists mainly of nerves, which are long fibers that connect the CNS to every other p

In [84]:
topic_label_tfidf_df= pd.DataFrame(topic_label_tfidf, columns=['terms', 'label'])
topic_label_tfidf_df

,terms,label
0,vmware server virtual oracle update virtualiza...,cloud computing
1,vmware server virtual oracle update virtualiza...,microsoft exchange server
2,vmware server virtual oracle update virtualiza...,vmware
3,vmware server virtual oracle update virtualiza...,web application
4,vmware server virtual oracle update virtualiza...,virtualization
...,...,...
1151,model datum estimate method predict parameter ...,parameter
1152,model datum estimate method predict parameter ...,regression analysis
1153,model datum estimate method predict parameter ...,estimation
1154,exposure concentration water soil metal study ...,ozone


In [85]:
# Write bhatia_topics_filtered_tfidf to folder
import csv
terms_labels={}
for terms, label in zip(topic_label_tfidf_df['terms'], topic_label_tfidf_df['label']):
    if terms not in terms_labels:
        terms_labels[terms]= label
    else:
        terms_labels[terms]= terms_labels[terms]+','+label
print(len(terms_labels))
#save to folder in csv file
with open('/data/bhatia_topics_filtered_tfidf/bhatia_topics_filtered_tfidf.csv','w') as csv_file:
    csv_writer= csv.writer(csv_file)
    for terms, labels in terms_labels.items():
        print(labels)
        print("---")
        print(terms)
        
        csv_writer.writerow([labels,terms.replace(',',' ')])

428
cloud computing,microsoft exchange server,vmware,virtualization,operating system
---
vmware server virtual oracle update virtualization application infrastructure management microsoft mac parallels os apple x hardware desktop allows users software machine intel hypervisor virtualized linux run product operating machines leopard running windows v xserves guest dtk licensing xserve systems lion restrictions enterprise demoed workloads macworld freebsd environment
web application
---
vmware server virtual oracle update virtualization application infrastructure management microsoft plesk web hosting dns windows linux accounts reseller software aps sql administrator panel cloudlinux automation swsoft platform packaging designed versions install extensions websites predetermine entries interface joomla systems coldfusion commercial centos opensuse drupal mail postgresql supports docker
conservative democrat,democratic party (united states),presidential nominee,bill clinton
---
obama mcca